In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
train_path = '/content/drive/My Drive/data/seg_train/'
test_path = '/content/drive/My Drive/data/seg_test/'

img_height = 180
img_width = 180
batch_size = 64

In [10]:
#validation_split

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 2400 images belonging to 6 classes.
Found 600 images belonging to 6 classes.


In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), input_shape=(180, 180, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 178, 178, 32)      896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 87, 87, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 41, 41, 128)       73856     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 18, 18, 128)      

In [12]:
history = model.fit(train_generator,
                    epochs = 15,
                    validation_data = validation_generator)

Epoch 1/15
38/38 [==============================] - 25s 627ms/step - loss: 1.6736 - accuracy: 0.2846 - val_loss: 1.2008 - val_accuracy: 0.5050
Epoch 2/15
38/38 [==============================] - 23s 608ms/step - loss: 1.0776 - accuracy: 0.5779 - val_loss: 1.0910 - val_accuracy: 0.5683
Epoch 3/15
38/38 [==============================] - 23s 610ms/step - loss: 1.0719 - accuracy: 0.5807 - val_loss: 1.0835 - val_accuracy: 0.5567
Epoch 4/15
38/38 [==============================] - 23s 609ms/step - loss: 0.9739 - accuracy: 0.6186 - val_loss: 1.0737 - val_accuracy: 0.5400
Epoch 5/15
38/38 [==============================] - 23s 608ms/step - loss: 0.9028 - accuracy: 0.6503 - val_loss: 1.1030 - val_accuracy: 0.5667
Epoch 6/15
38/38 [==============================] - 23s 613ms/step - loss: 0.8761 - accuracy: 0.6521 - val_loss: 0.9530 - val_accuracy: 0.6100
Epoch 7/15
38/38 [==============================] - 23s 609ms/step - loss: 0.7828 - accuracy: 0.7039 - val_loss: 0.9865 - val_accuracy: 0.6150

In [7]:
#VGG스타일, 필터 수가 2배씩 증가

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), input_shape=(180, 180, 3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(256, (3, 3), padding='valid', activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 178, 178, 32)      896       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 178, 178, 64)      18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 89, 89, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 89, 89, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 89, 89, 128)       73856     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 87, 87, 256)       295168    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 43, 43, 256)      

In [8]:
history = model.fit(train_generator,
                    epochs = 15,
                    validation_data = validation_generator)

Epoch 1/15
38/38 [==============================] - 783s 20s/step - loss: 4.9088 - accuracy: 0.1871 - val_loss: 1.6427 - val_accuracy: 0.2517
Epoch 2/15
38/38 [==============================] - 28s 721ms/step - loss: 1.6139 - accuracy: 0.2936 - val_loss: 1.4984 - val_accuracy: 0.3983
Epoch 3/15
38/38 [==============================] - 28s 722ms/step - loss: 1.4804 - accuracy: 0.3933 - val_loss: 1.3642 - val_accuracy: 0.4317
Epoch 4/15
38/38 [==============================] - 28s 722ms/step - loss: 1.3157 - accuracy: 0.4654 - val_loss: 1.2679 - val_accuracy: 0.4850
Epoch 5/15
38/38 [==============================] - 28s 727ms/step - loss: 1.2522 - accuracy: 0.5031 - val_loss: 1.2316 - val_accuracy: 0.4633
Epoch 6/15
38/38 [==============================] - 28s 725ms/step - loss: 1.2187 - accuracy: 0.5112 - val_loss: 1.1550 - val_accuracy: 0.5250
Epoch 7/15
38/38 [==============================] - 27s 719ms/step - loss: 1.1949 - accuracy: 0.5084 - val_loss: 1.1355 - val_accuracy: 0.5200
